In [1]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML 

In [2]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [3]:
dataset = pd.read_csv('./data/Online_Retail.csv',delimiter = ";") 

In [4]:
# посомтрим на датасет
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01 12 10 8:26,"2,55",17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01 12 10 8:26,"3,39",17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01 12 10 8:26,"2,75",17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01 12 10 8:26,"3,39",17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01 12 10 8:26,"3,39",17850.0,United Kingdom


In [5]:
transactions = []
for i in range(0, 7000): 
    transactions.append([str(dataset.values[i,j]) for j in [0, len(dataset.columns) - 1]])

In [6]:
import apyori
from apyori import apriori

In [7]:
%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 4, min_length = 2, max_length = 2))

Wall time: 12 ms


Визуализируем выход

In [8]:
import shutil, os 

In [9]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [10]:
import json #преобразовывать будем в json, используя встроенные в модуль методы

In [11]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

In [12]:
# и взгялнем на итоги
pd.set_option('display.max_colwidth', None)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

ValueError: Value must have type '<class 'int'>'

Итого мы видим:

1. Пары items
2. items_base - первый элемент пары
3. items_add - второй (добавленный алгоритмом) элемент пары
4. confidence - значение confidence для пары
5. lift - значение lift для пары
6. support - начение support для пары. При желании, по нему можно отсортировать 

### ECLAT Algorithm

#### Реализация в Python

In [ ]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = str(item).strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

Потестируем

In [ ]:
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.009, max_items = 4, min_items = 3)

In [ ]:
#обучим
model.fit(dataset)

In [ ]:
#и визуализируем результаты
model.transform()

Как видно, реализовать алгоритм своими силами довольно просто, хотя с эффективностью стоит поработать:)

### FP-Growth Algorithm

#### Реализация в Python

In [ ]:
import pyfpgrowth

In [ ]:
dataset

In [ ]:
# Group products by transaction
grouped_prods = dataset.groupby('InvoiceNo')['Description'].apply(lambda group_series: group_series.tolist()).reset_index()
groups_lists = grouped_prods['Description'].values.tolist()
# Convert table to list
# Set threshold of count to 2
data = list(filter(lambda x: len(x) > 2, groups_lists))

In [ ]:
#Сгенериуем паттерны
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [ ]:
#Сгенериуем правила
# use fp-growth algorithm
from mlxtend.frequent_patterns import fpgrowth

f_patterns = fpgrowth(df, min_support=0.01, use_colnames=True)
f_patterns

In [ ]:
# 2 способ
#Сгенериуем паттерны
patterns = pyfpgrowth.find_frequent_patterns(data, 20)
#Выучим правила
rules = pyfpgrowth.generate_association_rules(patterns, 50);
#Покажем
rules